<a href="https://colab.research.google.com/github/guillermohenrion/AI/blob/master/Reinforcement_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[q-learning](https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/)